### Instructions
1. Make sure you have changed the ```IP_ADDRESS``` and ```CAMERA_IP_ADDRESS``` variables.
2. Make sure your mosquitto is running
3. Make sure IP Webcam App is running - press Start server in app
4. Ideally, you should see "Connected" printed when runnning the 2nd block of code

In [1]:
import cv2
import urllib
import numpy as np
import base64
import requests
from requests.auth import HTTPBasicAuth
import os
import paho.mqtt.client as mqtt
import numpy as np
from PIL import Image
import json

IP_ADDRESS = "192.168.43.172" # Laptop IP - obtained through ipconfig in cmd prompt

URL = "http://192.168.43.1:8080/shot.jpg?"
CAMERA_IP_ADDRESS = "192.168.1.125:8080" # URL obtained from IP Webcam App

USERNAME = os.getenv('_USERNAME')
PASSWORD = os.getenv('_PASSWORD')

In [2]:
auth = HTTPBasicAuth(USERNAME, PASSWORD)

def setup(hostname):
    client = mqtt.Client()
    client.on_connect = on_connect
    client.on_message = on_message
    client.username_pw_set(USERNAME, PASSWORD)
    client.connect(hostname)
    client.loop_start()
    return client

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected.")
        client.subscribe("Group_25/ImageRec/Notify")
    else:
        print("Failed to Connect. Error code: %d." % rc)
        
def on_message(client, userdata, msg):
    if msg.payload == b'1':
        try:
            img_res = requests.get(f"http://{CAMERA_IP_ADDRESS}/shot.jpg?", auth=auth)
            frame = np.array(bytearray(img_res.content), dtype=np.uint8)
            frame = cv2.imdecode(frame, -1)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert to grayscale
            frame = frame.tobytes()
            # Just publish the jpg image to the client
            client.publish("Group_25/CNN/Input", frame)
        except:
            print("Your IP Camera isn't on!")

def main():
    client = setup(IP_ADDRESS)

if __name__ == "__main__":
    main()


Connected.
